In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision import utils
from torch.utils.data import sampler
from torch.utils.data import Dataset, DataLoader
import glob
import os.path as osp
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches
from torch.autograd import Variable
import sklearn
from sklearn.metrics import confusion_matrix
import scipy.ndimage
import cv2
import math
import dill
import pickle
import torchvision.models as models


In [8]:
def default_preprocessing(image, 
                          min_hw_ratio = 1, 
                          output_width = 299, 
                          output_height = 299):

    # Trim equal rows from top and bottom to get a square image
    r, c = image.shape
    image_hw_ratio = r/c
    r_to_keep = c * min_hw_ratio
    r_to_delete = r - r_to_keep
    remove_from_top = int(math.ceil(r_to_delete/2))
    remove_from_bottom = int(math.floor(r_to_delete/2))
    image_top_bottom_trimmed = image[remove_from_top:(r-remove_from_bottom),:]

    # resample to get the desired image size
    image_resampled = cv2.resize(image_top_bottom_trimmed, dsize=(output_width, output_height), interpolation=cv2.INTER_CUBIC)
    
    # Normalize pixel values to take the range [0,1]
    image_clean = image_resampled - np.mean(image_resampled)
    image_clean = image_clean / np.std(image_clean)
    image_clean = ((image_clean - np.min(image_clean)) / (np.max(image_clean) - np.min(image_clean)))  

    # Stack into three channels
    image_clean_stacked = np.dstack((image_clean, image_clean, image_clean))
    image_clean_stacked = np.moveaxis(image_clean_stacked, -1, 0)  
    
    # Implement ImageNet Standardization
    imagenet_mean = np.array([0.485, 0.456, 0.406]).reshape((3,1,1))
    imagenet_std = np.array([0.229, 0.224, 0.225]).reshape((3,1,1))
    image_clean_stacked = (image_clean_stacked - imagenet_mean) / imagenet_std
    
    return image_clean_stacked



In [9]:
# Initialize a model
pretrained_model = torchvision.models.densenet169(pretrained = True)

for param in pretrained_model.parameters():
    param.requires_grad = False
pretrained_model.aux_logits = False

# Modify the output layer of the densenet to fit our number of output classses
num_features = pretrained_model.classifier.in_features
num_features_knee = 14976

out_features = pretrained_model.classifier.out_features
pretrained_model.classifier = nn.Linear(num_features_knee, 5)

for param in pretrained_model.classifier.parameters():
    param.requires_grad = False

# Load the model's trained weights
pretrained_model.load_state_dict(torch.load('./DenseNet_classification_lr_0.0001_val_acc_0.6913822735084704_pretrained_True_epoch_6_currentLR_1e-05_DropOutProb_0.0', 
                                            map_location=lambda storage, 
                                            loc: storage))
pretrained_model.train(False)
pretrained_model = pretrained_model.double()

# Save it into Onnx format for export to iOS
from torch.autograd import Variable
import torch.onnx

# Create a sample image, which onnx needs
sample_input = np.load('L.npy').astype('float') 
print(sample_input)
sample_input = default_preprocessing(sample_input)

sample_input = sample_input.reshape((1,sample_input.shape))
sample_input = torch.from_numpy(sample_input)
sample_input = sample_input.double()

# Change the name of the first layer of the onnx model to "data"
input_names = [ "data" ] 

torch.onnx._export(pretrained_model, 
                  sample_input, 
                  "KneeDeep.onnx", 
                   input_names = input_names,
                   export_params=True,
                  verbose=True)



[[1382. 1393. 1437. ...  252.  255.  227.]
 [1418. 1352. 1413. ...  257.  249.  220.]
 [1389. 1429. 1439. ...  235.  243.  256.]
 ...
 [1596. 1650. 1535. ...  919.  905.  803.]
 [1634. 1587. 1512. ...  926.  920.  788.]
 [1578. 1518. 1582. ...  912.  859.  813.]]
(356.0, 356.0)


TypeError: slice indices must be integers or None or have an __index__ method

In [6]:
pwd

'/media/lukasz/mri/kevin/ai_data'

In [ ]:
/media/lukasz/mri/kevin/ai_data/AppDevelopment.ipynb